In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly
from retrieveData import DataHandler
import datetime as dt
import os
import plotly.express as px
from plotly.graph_objs import Figure
import plotly.io as pio
from typing import List, Union, Optional
from scipy import stats


file_path = '/Users/davidboktor/Documents/Tiingo_API.txt'
handler = DataHandler(file_path)


test = handler.test_connections()


Connection successful: {'message': 'You successfully sent a request'}


In [4]:
with open('./sp500_companies_2020.txt', 'r') as tickerlist:
    tickers = tickerlist.read().strip()
    ticker_list = tickers.split('\n')

print(f'Number of Tickers: {len(ticker_list)}')


start_date = "2015-01-01"
end_date = dt.date.today()

print(f"Historical data will be from {start_date} to {end_date}")


historical_data = handler.get_historical_data(start_date=start_date, end_date=end_date, tickers=ticker_list, frequency='monthly')
type(historical_data)

ERROR:retrieveData:Unexpected error for ABC: 'date'


Number of Tickers: 502
Historical data will be from 2015-01-01 to 2024-08-23


INFO:retrieveData:Successfully fetched data for ADSK
INFO:retrieveData:Successfully fetched data for ADBE
INFO:retrieveData:Successfully fetched data for AAPL
INFO:retrieveData:Successfully fetched data for ACN
INFO:retrieveData:Successfully fetched data for AAL
INFO:retrieveData:Successfully fetched data for ABBV
INFO:retrieveData:Successfully fetched data for ADP
INFO:retrieveData:Successfully fetched data for ABT
INFO:retrieveData:Successfully fetched data for AAP
INFO:retrieveData:Successfully fetched data for A
INFO:retrieveData:Successfully fetched data for ADM
INFO:retrieveData:Successfully fetched data for AEE
INFO:retrieveData:Successfully fetched data for ADI
INFO:retrieveData:Successfully fetched data for AEP
INFO:retrieveData:Successfully fetched data for ABMD
INFO:retrieveData:Successfully fetched data for AIG
INFO:retrieveData:Successfully fetched data for AKAM
INFO:retrieveData:Successfully fetched data for AFL
INFO:retrieveData:Successfully fetched data for ALGN
INFO:re

pandas.core.frame.DataFrame

In [5]:
# Drop the rows where all the values are NaN
historical_data.dropna(axis=0, how='all', inplace=True)
historical_data.reset_index(inplace=True, names='date')
historical_data.head()

/var/folders/nv/59bkjyks06gdx0l5k1v5pqk40000gn/T/ipykernel_12526/370928360.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  historical_data.reset_index(inplace=True, names='date')


,date,A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,2015-01-30,34.915927,46.496564,143.995752,26.118257,40.485023,51.74,37.370819,71.526434,70.13,...,27.877293,33.912799,56.949214,45.801297,30.212795,43.403084,100.981221,83.460,18.849482,39.918517
1,2015-02-27,39.020421,45.471646,140.309823,28.749564,40.585649,60.79,39.549948,76.633641,79.10,...,26.205993,37.527326,58.113787,48.534268,31.753908,48.705370,108.448968,91.050,21.064225,43.057441
2,2015-03-31,38.503726,50.104248,135.618881,27.847643,39.270808,71.58,38.681636,79.749037,73.94,...,26.102111,37.465326,55.790285,46.659905,31.149072,47.270210,106.059741,90.715,21.273016,43.244281
3,2015-04-30,38.336923,45.932059,129.557753,28.008781,43.737801,63.22,38.956685,79.709467,76.06,...,25.427250,38.404174,57.345850,46.760761,32.928002,51.888009,99.128674,92.080,22.324849,41.571615
4,2015-05-29,38.170120,40.305298,138.817055,29.277924,45.043306,59.72,40.786189,82.625982,79.09,...,25.532228,42.296856,56.391973,47.709729,32.653772,54.393072,103.166766,109.640,22.800638,46.578552


In [6]:
def plotData(df: pd.DataFrame, x: str, y: str, title: str = None) -> Figure:
    fig = px.line(df, x=x, y=y)

    if title is None:
        fig.update_layout(title=f"Historical Data of {y}")
    else:
        fig.update_layout(title=title)
    
    # Update layout for size
    fig.update_layout(
        width=1000, 
        height=600,
        xaxis_title=x,
        yaxis_title=y
    )
    
    return fig

In [7]:
plotData(historical_data, 'date', 'AAPL')


In [8]:
if 'date' in historical_data.columns:
    historical_data.set_index('date', inplace=True)

# Fill NaN values
historical_data.fillna(method='bfill', inplace=True)
historical_data.fillna(method='ffill', inplace=True)

historical_returns = handler.get_returns(historical_data)
historical_returns.dropna(inplace=True)
historical_returns.head()

/var/folders/nv/59bkjyks06gdx0l5k1v5pqk40000gn/T/ipykernel_12526/478181316.py:5: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/var/folders/nv/59bkjyks06gdx0l5k1v5pqk40000gn/T/ipykernel_12526/478181316.py:6: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



,A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-02-27,0.117554,-0.022043,-0.025597,0.100746,0.002486,0.174913,0.058311,0.071403,0.127905,0.131119,...,-0.059952,0.106583,0.020449,0.059670,0.051009,0.122164,0.073952,0.090942,0.117496,0.078633
2015-03-31,-0.013242,0.101879,-0.033433,-0.031372,-0.032397,0.177496,-0.021955,0.040653,-0.065234,0.076187,...,-0.003964,-0.001652,-0.039982,-0.038619,-0.019048,-0.029466,-0.022031,-0.003679,0.009912,0.004339
2015-04-30,-0.004332,-0.083270,-0.044692,0.005786,0.113748,-0.116792,0.007111,-0.000496,0.028672,-0.018413,...,-0.025855,0.025059,0.027882,0.002162,0.057110,0.097689,-0.065351,0.015047,0.049444,-0.038679
2015-05-29,-0.004351,-0.122502,0.071469,0.045312,0.029848,-0.055362,0.046963,0.036589,0.039837,0.105402,...,0.004129,0.101361,-0.016634,0.020294,-0.008328,0.048278,0.040736,0.190704,0.021312,0.120441
2015-06-30,-0.061024,-0.057470,0.040009,-0.037266,0.009010,0.100636,0.009877,0.007705,0.024276,-0.055547,...,-0.045579,-0.068747,-0.023474,-0.007945,0.013672,-0.000333,-0.042598,0.012860,0.098857,-0.031143


In [9]:
plotData(historical_returns.reset_index(), 'date', 'AAPL', 'Historical Returns for AAPL')

In [10]:
historical_log_returns = handler.get_returns(historical_data, log=True)
historical_log_returns.dropna(inplace=True)
historical_log_returns.head()

,A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-02-27,0.111142,-0.022289,-0.025931,0.095988,0.002482,0.161194,0.056674,0.068969,0.120362,0.123207,...,-0.061824,0.101277,0.020243,0.057958,0.049750,0.115259,0.071345,0.087041,0.111091,0.075695
2015-03-31,-0.013330,0.097017,-0.034004,-0.031874,-0.032933,0.163390,-0.022199,0.039849,-0.067459,0.073424,...,-0.003972,-0.001653,-0.040803,-0.039385,-0.019231,-0.029909,-0.022277,-0.003686,0.009863,0.004330
2015-04-30,-0.004342,-0.086942,-0.045722,0.005770,0.107731,-0.124195,0.007085,-0.000496,0.028269,-0.018584,...,-0.026195,0.024750,0.027501,0.002159,0.055539,0.093207,-0.067584,0.014935,0.048261,-0.039447
2015-05-29,-0.004360,-0.130680,0.069030,0.044316,0.029412,-0.056954,0.045893,0.035936,0.039064,0.100209,...,0.004120,0.096547,-0.016774,0.020091,-0.008363,0.047149,0.039928,0.174545,0.021088,0.113723
2015-06-30,-0.062966,-0.059187,0.039229,-0.037978,0.008970,0.095888,0.009828,0.007676,0.023986,-0.057150,...,-0.046651,-0.071225,-0.023754,-0.007977,0.013580,-0.000333,-0.043532,0.012778,0.094271,-0.031639


In [11]:
plotData(historical_log_returns.reset_index(), 'date', 'AAPL', 'Historical Log-returns for AAPL')

In [12]:
signal_df = pd.DataFrame(0, index=historical_log_returns.index, columns=historical_log_returns.columns)
signal_df.head()

,A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-02-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-03-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-04-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-05-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-06-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
def get_top_n(df_returns: pd.DataFrame, n: int) -> pd.DataFrame:
    """
    Create a DataFrame marking the top n performing tickers for each date.

    :param df_returns: DataFrame of returns, with dates as index and tickers as columns
    :param n: Number of top performers to select for each date
    :return: DataFrame with 1 for top n performers and 0 for others
    """
    # Initialize a DataFrame with zeros, same shape as input
    top_tickers_df = pd.DataFrame(0, index=df_returns.index, columns=df_returns.columns)
    
    # Get the indices of top n values for each row
    top_n_indices = df_returns.apply(lambda x: x.nlargest(n).index, axis=1)
    
    # Set values to 1 for top n performers
    for date, tickers in top_n_indices.items():
        top_tickers_df.loc[date, tickers] = 1
    
    return top_tickers_df


In [14]:
longs_df = get_top_n(historical_log_returns, 50)
longs_df.head()

,A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-02-27,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-03-31,0,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2015-04-30,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2015-05-29,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,1
2015-06-30,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
shorts_df = get_top_n(-1*historical_log_returns, 50)
shorts_df.head()

,A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-02-27,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2015-03-31,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2015-04-30,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-05-29,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-06-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
lookahead_returns = historical_log_returns.shift(-1)
lookahead_returns

,A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-02-27,-0.013330,0.097017,-0.034004,-0.031874,-0.032933,0.163390,-0.022199,0.039849,-0.067459,0.073424,...,-0.003972,-0.001653,-0.040803,-0.039385,-0.019231,-0.029909,-0.022277,-0.003686,0.009863,0.004330
2015-03-31,-0.004342,-0.086942,-0.045722,0.005770,0.107731,-0.124195,0.007085,-0.000496,0.028269,-0.018584,...,-0.026195,0.024750,0.027501,0.002159,0.055539,0.093207,-0.067584,0.014935,0.048261,-0.039447
2015-04-30,-0.004360,-0.130680,0.069030,0.044316,0.029412,-0.056954,0.045893,0.035936,0.039064,0.100209,...,0.004120,0.096547,-0.016774,0.020091,-0.008363,0.047149,0.039928,0.174545,0.021088,0.113723
2015-05-29,-0.062966,-0.059187,0.039229,-0.037978,0.008970,0.095888,0.009828,0.007676,0.023986,-0.057150,...,-0.046651,-0.071225,-0.023754,-0.007977,0.013580,-0.000333,-0.043532,0.012778,0.094271,-0.031639
2015-06-30,0.059618,0.004123,0.089535,-0.033441,0.048418,0.164206,0.037092,0.063356,0.012025,-0.095653,...,0.074530,-0.056120,-0.049145,0.098919,-0.070980,-0.021490,-0.048392,-0.031281,-0.017323,0.015638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-29,-0.058352,-0.127685,-0.150167,-0.006729,-0.103511,0.000000,-0.065027,-0.137420,-0.086406,0.014157,...,-0.000372,0.000000,0.017313,-0.100718,0.011233,0.018579,-0.092828,0.042604,-0.062268,-0.057911
2024-04-30,-0.049590,-0.161083,-0.032589,0.122424,-0.008645,0.000000,-0.036322,-0.063876,-0.039825,0.156050,...,0.031510,0.000000,-0.000535,-0.068969,0.078576,-0.022561,-0.043587,-0.007082,0.066467,0.062794
2024-05-31,-0.005999,-0.014893,-0.109237,0.091259,0.061806,0.000000,0.016691,0.072150,0.222406,-0.022946,...,-0.027353,0.000000,-0.018419,-0.110890,-0.038971,-0.036834,-0.056991,-0.010978,0.004159,0.022164


In [17]:
def portfolio_returns(df_long: pd.DataFrame, df_short: pd.DataFrame, lookahead_returns: pd.DataFrame, n_stocks: int) -> pd.DataFrame:
    """
    Compute expected returns for the portfolio, assuming equal investment in each long/short stock.
    
    Parameters
    ----------
    df_long : DataFrame
        Top stocks for each ticker and date marked with a 1
    df_short : DataFrame
        Bottom stocks for each ticker and date marked with a 1
    lookahead_returns : DataFrame
        Lookahead returns for each ticker and date
    n_stocks: int
        The number number of stocks chosen for each month
    
    Returns
    -------
    portfolio_returns : DataFrame
        Expected portfolio returns for each ticker and date
    """

    long_returns = df_long * lookahead_returns / n_stocks
    short_returns = df_short * lookahead_returns / n_stocks
    
    portfolio_returns = long_returns - short_returns
    
    return portfolio_returns

In [18]:
expected_portfolio_returns = portfolio_returns(longs_df, shorts_df, lookahead_returns, 2*50)
expected_portfolio_returns

,A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-02-27,0.0,0.000000,0.000000,0.000000,0.000000,0.001634,0.00000,0.000000,0.000000,0.000000,...,0.00004,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
2015-03-31,0.0,-0.000869,0.000000,0.000000,0.000000,-0.001242,0.00000,0.000000,-0.000283,-0.000186,...,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
2015-04-30,0.0,0.001307,0.000000,0.000000,0.000294,0.000570,0.00000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.0,0.000000,0.0,0.000471,0.0,0.000000,0.000000,0.000000
2015-05-29,0.0,0.000592,0.000000,0.000000,0.000000,-0.000959,0.00000,0.000000,0.000000,-0.000571,...,0.00000,-0.000712,0.0,0.000000,0.0,0.000000,0.0,0.000128,0.000000,-0.000316
2015-06-30,0.0,0.000000,0.000000,0.000000,0.000000,0.001642,0.00000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,-0.000173,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-29,0.0,0.000000,-0.001502,0.000067,0.000000,0.000000,0.00065,0.001374,0.000864,0.000000,...,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000579
2024-04-30,0.0,0.000000,0.000326,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.0,-0.000071,0.000000,0.000000
2024-05-31,0.0,0.000149,0.000000,0.000913,0.000000,0.000000,0.00000,-0.000722,0.000000,-0.000229,...,0.00000,0.000000,0.0,0.001109,0.0,0.000000,0.0,0.000000,0.000000,0.000000


In [19]:
expected_portfolio_returns_by_date = expected_portfolio_returns.T.sum().dropna()
portfolio_ret_mean = expected_portfolio_returns_by_date.mean()
portfolio_ret_ste = expected_portfolio_returns_by_date.sem()
portfolio_ret_annual_rate = (np.exp(portfolio_ret_mean * 12) - 1) * 100

print("""
Mean:                       {:.6f}
Standard Error:             {:.6f}
Annualized Rate of Return:  {:.2f}%
""".format(portfolio_ret_mean, portfolio_ret_ste, portfolio_ret_annual_rate))


Mean:                       -0.000701
Standard Error:             0.002392
Annualized Rate of Return:  -0.84%



In [20]:
expected_portfolio_returns_df = pd.DataFrame(data=expected_portfolio_returns_by_date, columns=['returns'])
expected_portfolio_returns_df.reset_index(inplace=True)

expected_portfolio_returns_df['date'] = pd.to_datetime(expected_portfolio_returns_df['date'])
expected_portfolio_returns_df = expected_portfolio_returns_df.sort_values(by='date')

expected_portfolio_returns_df['cumulative_returns'] = (1 + expected_portfolio_returns_df['returns']).cumprod() - 1

plotData(expected_portfolio_returns_df, 'date', 'cumulative_returns')

In [21]:

def analyze_alpha(expected_portfolio_returns_by_date):
    """
    Perform a t-test with the null hypothesis being that the expected mean return is zero.
    
    Parameters
    ----------
    expected_portfolio_returns_by_date : Pandas Series
        Expected portfolio returns for each date
    
    Returns
    -------
    t_value
        T-statistic from t-test
    p_value
        Corresponding p-value
    """
    # TODO: Implement Function
    
    t_test = stats.ttest_1samp(expected_portfolio_returns_by_date,
                               popmean=0,
                              )
                    
    
    return (t_test[0], t_test[1] / 2)

In [22]:
analyze_alpha(expected_portfolio_returns_by_date)

(np.float64(-0.2931185058116381), np.float64(0.3849845907064786))